In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import cmath
from numba import njit, f8,i8,c16
import numba
import matplotlib.animation as animation
import copy
import matplotlib
import random
import concurrent
import func_for_paralell
import Another_RungeKutta

import sys
sys.path.append('../../lb')
import RungeKutta_acceralated

#パラメーター
nu = 0.001
f = (1 + 1j) * 5 * 0.001
Dimention = 14 #uの次元数
q = 2
k_0 = 2 ** (-4)
k_n = np.zeros((Dimention+4,1))
k_n[2:-2,0] = [k_0 * q ** (i+1) for i in range(Dimention)]
#print(k_n)

c_n_1 = copy.deepcopy(k_n)[2:-2, 0:]
c_n_1[-2:, 0:] = np.zeros((2,1))
#print(c_n_1)

beta = 0.5
c_n_2 = copy.deepcopy(k_n)[1:-3, 0:] * -beta
c_n_2[-1, 0] = np.zeros((1, 1))
# print(c_n_2)

c_n_3 = copy.deepcopy(k_n)[:-4, 0:] * (beta - 1)
#print(c_n_3)

#parameter = [nu,f,k_n,c_n_1,c_n_2,c_n_3]
parameter = {
    'nu' : nu,
    'f' : f,
    'k_n' : k_n,
    'c_n_1' : c_n_1,
    'c_n_2' : c_n_2,
    'c_n_3' : c_n_3
}
#print(parameter)
# print(k_n)

runge_para = {
    'start' :  np.load("14次元初期値.npy"),
    'step' : 0.01,
    'period' : np.array([0,5000]),
    'nu' : nu,
    'f' : f,
    'k_n' : k_n,
    'c_n_1' : c_n_1,
    'c_n_2' : c_n_2,
    'c_n_3' : c_n_3
}



# コンテスト

In [ ]:
%%time
runge_para["period"]=(0,5000)

#刻み幅
runge_para["step"] = 0.01
skip= 10
competitives = 10

plt.figure(figsize=(60,5))

runge_para['start'] = np.load("beta0.473_nu0.0001305_0.0114dim.npy")

#nu
runge_para["nu"] = 0.00013095454545454546
#betaの値に従いc_n_2,c_n_3を更新
beta= 0.4745151515151515

#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

#摂動を与えた初期値のリスト
starts = np.zeros((14,competitives))
starts = starts.astype(np.complex128)

lmodel = Long_laminar() #インスタンス化
starts[:,0:1] = runge_para["start"]
for i in range(competitives - 1):
    starts[:,i+1:i+2] = Long_laminar.perturbator(runge_para["start"])

#ラミナーから出た極大値の時間
times = np.zeros((competitives,1))

for i in tqdm(range(competitives)):
    cut_arr = np.zeros((Dimention+1,1))
    
    runge_para['start'] = starts[:,i:i+1]
    #インスタンス化&実行
    model = RungeKutta_acceralated.RungeKutta()
    model.get_arr_latter(**runge_para, latter = 1)
    #プロット
    plt.subplot(1,1,1)
    plt.ylabel(f"u{1}")
    plt.yscale=("log")
    plt.plot(np.real(model.arr_latter[-1,::skip]),abs(model.arr_latter[0,::skip]),lw=0.5)
    
#     #極大抽出
#     cut_arr = loc_max_7(model.arr_latter,cut_arr)
    
#     for j in range(len(cut_arr[0,:])):
#         if not ((0.52 < cut_arr[0,j] < 0.68 or 0.78 < cut_arr[0,j] < 1) 
#             and (0.02 < cut_arr[1,j] < 0.13 or 0.19 < cut_arr[1,j] < 0.34)
#             and (0.12 < cut_arr[2,j] < 0.26 or 0.36 < cut_arr[2,j] < 0.48)
#             and (0.17 < cut_arr[3,j] < 0.24 or 0.25 < cut_arr[3,j] < 0.32 or 0.34 < cut_arr[3,j] < 0.42)
#             and (0.08 < cut_arr[4,j] < 0.2 or 0.28 < cut_arr[4,j] < 0.35)
#             and (0.05 < cut_arr[5,j] < 0.115 or 0.12 < cut_arr[5,j] < 0.155)
#             and (0.03 < cut_arr[6,j] < 0.07 or 0.1 < cut_arr[6,j] < 0.22)
#             and (0.02 < cut_arr[7,j] < 0.08 or 0.104 < cut_arr[7,j] <0.144)
#             and (0.014 < cut_arr[8,j] < 0.078)
#             and (0 < cut_arr[9,j] < 0.036 or 0.048 < cut_arr[9,j] < 0.076 or 0.084 < cut_arr[9,j] < 0.1)
#             and (0 < cut_arr[10,j] < 0.014 or 0.02 < cut_arr[10,j] < 0.038 or 0.046 < cut_arr[10,j] < 0.068)
#             and (0 < cut_arr[11,j] < 0.002 or 0.003 < cut_arr[11,j] < 0.0095 or 0.0135 < cut_arr[11,j] < 0.0225)
#             and (0 < cut_arr[12,j] < 0.0006 or 0.001 < cut_arr[12,j] < 0.0024)
#             and (0 < cut_arr[13,j] < 0.000005 or 0.00001 < cut_arr[13,j] < 0.000045)):
#             times[i,0] = cut_arr[-1,j]
#             break
            
#plt.savefig(fname=f"../laminar/contest_a=-8_b=-15_nu{runge_para['nu']}_beta{beta}_{runge_para['step']}_{Dimention}dim.png")

# print(np.mean(times))
# print(np.std(times))
# print(np.min(times))
# print(np.max(times))

# ラミナー読み込み

In [ ]:
laminar = np.load("../laminar_initials/beta0.423_nu0.00017584784643038092_step0.01_1500check_500pro_20000period.npz")["laminar"][:,:500000:200]
plt.scatter(range(laminar.shape[1]),abs(laminar[0,:]))

# アップデート

In [ ]:
#並列ver.
#ある程度持続すれば更新
runge_para["period"]=np.array([0,3000])
competitives = 100

#刻み幅
runge_para["step"] = 0.01
runge_para['start'] = laminar[:,:1]


plus = 1E-4
#nu
runge_para["nu"] = 0.00011871562745803017
beta= 0.4671099999999965

skip=100
epsilon=1E-2

model = RungeKutta_acceralated.RungeKutta()
lmodel = RungeKutta_acceralated.Long_laminar()

while True:
    if beta > 0.5:
        break
    #betaの値に従いc_n_2,]}}c_n_3を更新
    #c_n_2
    runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
    runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

    #c_n_3
    runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)
    for _ in range(4):
        with concurrent.futures.ProcessPoolExecutor(max_workers = 8) as executor: 
            task1 = executor.submit(func_for_paralell.competition2, runge_para, competitives, laminar, skip, epsilon)
            task2 = executor.submit(func_for_paralell.competition2, runge_para, competitives, laminar, skip, epsilon)
            task3 = executor.submit(func_for_paralell.competition2, runge_para, competitives, laminar, skip, epsilon)
            task4 = executor.submit(func_for_paralell.competition2, runge_para, competitives, laminar, skip, epsilon)
            # task5 = executor.submit(func_for_paralell.competition2, runge_para, competitives, laminar, skip, epsilon)
            # task6 = executor.submit(func_for_paralell.competition2, runge_para, competitives, laminar, skip, epsilon)
            # task7 = executor.submit(func_for_paralell.competition2, runge_para, competitives, laminar, skip, epsilon)
            
            times1, starts1, nus1 = task1.result()
            times2, starts2, nus2 = task2.result()
            times3, starts3, nus3 = task3.result()
            times4, starts4, nus4 = task4.result()
            # times5, starts5, nus5 = task5.result()
            # times6, starts6, nus6 = task6.result()
            # times7, starts7, nus7 = task7.result()
        times = np.concatenate([times1, times2, times3, times4])
        starts = np.concatenate((starts1, starts2, starts3, starts4), axis=1)
        nus = np.concatenate([nus1, nus2, nus3, nus4])

        # times = np.concatenate([times1, times2, times3, times4, times5, times6, times7])
        # starts = np.concatenate((starts1, starts2, starts3, starts4, starts5, starts6, starts7), axis=1)
        # nus = np.concatenate([nus1, nus2, nus3, nus4, nus5, nus6, nus7])
        runge_para['nu'] = nus[np.argmax(times)]
        runge_para['start'] = starts[:, np.argmax(times):np.argmax(times)+1]
        print(max(times))
        if max(times) > runge_para["period"][1]*0.99:
            break
    if max(times) > 1000:
        np.save(f"../laminar_initials/beta{beta}_nu{runge_para['nu']}_step{runge_para['step']}_{int(max(times))}period_laminar", runge_para["start"])
    print(f"beta={beta}, nu = {runge_para['nu']} time = {max(times)}")
    model.get_arr_latter(**runge_para,latter = 1)
    plt.clf()
    plt.figure(figsize=(40,5))
    plt.plot(abs(model.arr_latter[-1,:]),abs(model.arr_latter[0,:]))
    plt.show()
    laminar = model.arr_latter[:-1,:round(model.arr_latter.shape[1]/10):10]
    beta += plus

# ラミナーに留まる(長時間)

## stagger and step

In [ ]:
%%time
runge_para["period"]=np.array([13000,20000])

#刻み幅
runge_para["step"] = 0.01
skip= 100

check =  3000
progress =  500

runge_para['start'] = lmodel.arr[:-1,-50001:-50000]
#nu
runge_para["nu"] = 0.00011871562745803017
#betaの値に従いc_n_2,]}}c_n_3を更新
beta= 0.46711

#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))
#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)
lmodel = Another_RungeKutta.Long_laminar()
lmodel.get_laminar(**runge_para, check_sec = check, progress_sec = progress, laminar = laminar, skip = 100, epsilon=1E-1)

np.savez_compressed(f"../laminar/beta{beta}_nu{runge_para['nu']}_step{runge_para['step']}_{check}check_{progress}pro_{round(abs(lmodel.arr[-1,-1]))}period",start = lmodel.arr[:-1,0:1], laminar = lmodel.arr)

In [ ]:
skip=1
plt.figure(figsize=(30,40))

for i in range(len(lmodel.arr[:-1,0])):
    plt.subplot(len(lmodel.arr[:-1,0]),1,i+1)
    plt.ylabel(f"u{i+1}")
    #plt.yscale=("log")
    plt.plot(np.real(lmodel.arr[-1,::skip]),abs(lmodel.arr[i,::skip]),lw=0.5)
    for j in range(len(lmodel.perturbated_time)):
        plt.vlines(x=lmodel.perturbated_time[j], ymin=np.min(abs(lmodel.arr[i,:])), ymax=np.max(abs(lmodel.arr[i,:])),colors="red")
#plt.savefig(fname=f"../laminar/nu{runge_para['nu']}_beta{beta}_{runge_para['step']}_{check}check_{progress}progress.png")

In [ ]:
a, b = 3, 4
fig = plt.figure()
ax = plt.subplot(1,1,1)
ax.set_xlabel(f"u{a+1}")
ax.set_ylabel(f'u{b+1}')
ax.set_xscale("log")
ax.set_yscale("log")
ax.plot(np.abs(lmodel.arr[a,:]),np.abs(lmodel.arr[b,:]),lw=0.1)
#plt.savefig(f"../laminar/nu{runge_para['nu']}_beta{beta}_{runge_para['step']}_{check}check_{progress}progress_u{a+1}&{b+1}.png")

## 初期値の調整

In [ ]:
e = int(len(zenkai[0,:])//100*50)
runge_para['start'] = zenkai[:-1,e:e+1]

In [ ]:
d = 46
runge_para['start'] = model.arr_latter[:-1,int(len(model.arr_latter[0,:])//100*d):int(len(model.arr_latter[0,:])//100*d)+1]

In [ ]:
runge_para['start'] = np.load("beta0.47

In [ ]:
# 縦軸に各変数、横軸時間　×次元数
runge_para["period"]=(0,10000)

#刻み幅
runge_para["step"] = 0.01

skip=100

#nu
runge_para["nu"] = 0.0001307878787878788
#betaの値に従いc_n_2,c_n_3を更新
beta = 0.4739595959595959


#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

#インスタンス化
model = RungeKutta_acceralated.RungeKutta()
model.get_arr_latter(**runge_para,latter=1)

plt.figure(figsize=(20,30))
for i in range(Dimention):
    plt.subplot(Dimention,1,i+1)
    plt.ylabel(f"u{i+1}")
    #plt.yscale=("log")
    plt.plot(np.real(model.arr_latter[-1,::skip]),abs(model.arr_latter[i,::skip]),lw=1)
    if runge_para["period"][1] == 100000:
        plt.xticks(np.arange(0,100000,5000))


#plt.savefig(fname=f"../attracter/nu{runge_para['nu']}_beta{beta}_{runge_para['step']}_time{runge_para['period']}_{Dimention}dim.png")

In [ ]:
np.savez_compressed("beta0.473_nu0.0001305_step0.01_300period_laminar.npz", laminar=model.arr_latter)

In [ ]:
a, b = 3, 4
fig = plt.figure()
ax = plt.subplot(1,1,1)
ax.set_xlabel(f"u{a+1}")
ax.set_ylabel(f'u{b+1}')
ax.set_xscale("log")
ax.set_yscale("log")
ax.plot(np.abs(model.arr_latter[a,:30000]),np.abs(model.arr_latter[b,:30000]),lw=0.1)

In [ ]:
#lmodel.perturbated_time

In [ ]:
#時間を指定して区切る
@njit
def trimmer(array,period,trimmered):
    for i in range(len(array[0,:])):
        if (array[-1,i] > period[0] and array[-1,i] < period[1]):
            trimmered = np.append(trimmered,array[:,i:i+1],axis = 1)
    return trimmered

In [ ]:
#c4が極大値を取る時の全変数(+時間)をndarrayで返す関数
@njit(f8[:,:](c16[:,:],f8[:,:]),cache=True)
def loc_max_7(arr,z_list):
    for j in range(len(arr[0,:]) - 3):
        #7点とって極大判定
        if (np.abs(arr[3,j+1]) - np.abs(arr[3,j]) > 0
        and np.abs(arr[3,j+2]) - np.abs(arr[3,j+1]) > 0
        and np.abs(arr[3,j+3]) - np.abs(arr[3,j+2]) > 0
        and np.abs(arr[3,j+4]) - np.abs(arr[3,j+3]) < 0
        and np.abs(arr[3,j+5]) - np.abs(arr[3,j+4]) < 0
        and np.abs(arr[3,j+6]) - np.abs(arr[3,j+5]) < 0):
            z_list = np.append(z_list,np.abs(arr[:,j+3:j+4]),axis=1)
    return z_list

In [ ]:
nu = 0.00017795206429227755
beta = 0.41676767676767673
step = 0.005
time = (0, 999945)
cut_array = np.load(f"../numpy_array/nu{nu}_beta{beta}_{step}_time{time}_14dim.npz")["array"]
step_number = int((runge_para["period"][1] - runge_para["period"][0]) // runge_para["step"]) + 1
cut_arr = np.empty((Dimention+1,1))
cut_arr[:,:] = np.nan
cut_arr = loc_max_7(lmodel.arr, cut_arr)[:,1:]

In [ ]:
#ポアンカレ写像のプロット(ラミナーの時間、出したい変数を入力)

#プロットする変数
var1 = 4
var2 = 3

#プロットする期間たち
periods = np.array([[10000,14000],
                    [87000,94000],
                    [135000,137000],
                    [160000,163000],
                    [175000,181000],
                    [189000,192000],
                    [194000,197000],
                    [206000,207000],
                    [228000,229000],
                    [234000,236000],
                    [262000,266000],
                    [275000,282000],
                    [288000,296000],
                    [309000,310000],
                    [322000,325000],
                    [338000,341000],
                    [346000,348000],
                    [368000,371000],
                    [380000,382000],
                    [405000,409000],
                    [412000,416000],
                    [420000,422000],
                    [459000,465000],
                    [480000,486000],
                    [487000,489000],
                    [522000,524000],
                    [538000,541000],
                    [605000,606000],
                    [624000,629000],
                    [640000,643000],
                    [646000,650000],
                    [652000,653000],
                    [659000,662000],
                    [681000,691000],
                    [707000,711000],
                    [723000,724000],
                    [726000,727000],
                    [759000,760000],
                    [763000,768000],
                    [773000,774000],
                    [793000,795000],
                    [820000,824000],
                    [825000,827000],
                    [832000,833000],
                    [838000,840000],
                    [843000,850000],
                    [851000,859000],
                    [872000,875000],
                    [896000,897000],
                    [904000,908000],
                    [909000,915000],
                    [921000,923000],
                    [926000,927000],
                    [942000,943000],
                    [977000,983000],
                    [992000,993000]
                   ])

#プロットの設定
plt.figure(figsize=(10,10))
plt.xlabel(f"|u{var1}|")
plt.ylabel(f"|u{var2}|")
#plt.xticks(np.arange(-0.6,0.6,step=0.02))
#plt.xticks(rotation=90)
#plt.yticks(np.arange(-0.4,0.4,step=0.02))
plt.grid(False)
plt.title(f"U_{4} is local maximum \n beta = {beta} \n nu = {nu}")

plt.scatter(cut_array[var1-1,:],cut_array[var2-1,:],s=5,c="blue",label="all")


for i in range(len(periods[:,0])):
    period = periods[i,:]
    trimmered = np.zeros((15,1))
    trimmered = trimmer(cut_array,period,trimmered)
    trimmered = np.delete(trimmered,0,1)
    if i == 0:
        plt.scatter(trimmered[var1-1,:],trimmered[var2-1,:],c="red",s=1,label="laminar")
    else:
        plt.scatter(trimmered[var1-1,:],trimmered[var2-1,:],c="red",s=5)
        
plt.scatter(cut_arr[var1-1,:], cut_arr[var2-1, :], s=5, c = "yellow", label = "long laminar", alpha =0.5)
plt.legend()



plt.savefig(fname=f"../poincare/beta{beta}_nu{nu}_{step}step_time{time}_{len(cut_array[:,0])}dim_u4_is_loc_max_U{var1}andU{var2}.png")